In [7]:
# coding=utf-8
# 驾考宝典题库爬虫 lsido.com
# 爬取各类车型科目一、科目四所有题，带更新

import requests
import re
from random import choice
import json
import random
from bs4 import BeautifulSoup
import MySQLdb
import sys
stdo = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout= stdo
print '1'
sys.setdefaultencoding('utf-8')

#get请求
def getHTMLText(url):
    try:
        r = requests.get(url, timeout = 30)
        r.raise_for_status()
        r.encoding = 'utf-8'
        return r.text
    except:
        return ""
#异常处理数据库链接
try:
    conn = MySQLdb.connect(host='localhost',port= 3306,user = 'root',passwd='123456',db='examination',charset='utf8')
    cur = conn.cursor()
    print '已连接数据库'
except Exception, e:
    print '发生了一个错误: %s, 你可以在这里删除错误的文档' % e
#python获取控制台参数
if sys.argv[1] == '小车':
    carType = 'car'
    chexing = 'C1C2C3C4'
elif sys.argv[1] == '货车':
    carType = 'truck'
    chexing = 'A2B2'
elif sys.argv[1] == '客车':
    carType = 'bus'
    chexing = 'A1A3B1'
elif sys.argv[1] == '摩托车':
    carType = 'moto'
    chexing = 'moto'
else:
    carType = 'car'
    chexing = 'C1C2C3C4'
kemu = sys.argv[2]
if sys.argv[2] == '科目一':
    r = getHTMLText('http://api2.jiakaobaodian.com/api/open/exercise/sequence.htm?_r=17801703702540802070&cityCode=511300&page=1&limit=25&course=kemu1&carType='+str(carType)+'&_=0.7121756361682074')
    pjson = json.loads(r)
    jlist = pjson['data']
else:
    r = getHTMLText('http://api2.jiakaobaodian.com/api/open/exercise/sequence.htm?_r=17801703702540802070&cityCode=511300&page=1&limit=25&course=kemu3&carType='+str(carType)+'&_=0.7121756361682074')
    pjson = json.loads(r)
    jlist = pjson['data']
a=0
data = jlist

#循环读取ID，从驾考宝典内获取单个id下的记录
for i in data:
    #各种异常傻瓜式处理
    try:
        problem = getHTMLText('http://api2.jiakaobaodian.com/api/open/question/question-list.htm?_r=19604815519963578102&page=1&limit=25&questionIds='+str(i))
        pjson = json.loads(problem)
        jlist = pjson['data']
        print jlist
    except:
        problem = getHTMLText('http://api2.jiakaobaodian.com/api/open/question/question-list.htm?_r=19604815519963578102&page=1&limit=25&questionIds='+str(i))
        pjson = json.loads(problem)
        jlist = pjson['data']

    if len(jlist) == 0:
        try:
            problem = getHTMLText('http://api2.jiakaobaodian.com/api/open/question/question-list.htm?_r=19604815519963578102&page=1&limit=25&questionIds='+str(i))
            pjson = json.loads(problem)
            jlist = pjson['data']
        except:
            problem = getHTMLText('http://api2.jiakaobaodian.com/api/open/question/question-list.htm?_r=19604815519963578102&page=1&limit=25&questionIds='+str(i))
            pjson = json.loads(problem)
            jlist = pjson['data']
    #异常处理获取某个题end

    #开始插入数据库
    for k in jlist:
        #数据库读取当前id题、判断数据库与驾考宝典是否一致，若无此题则插入本地数据库
        idCount=cur.execute("select * from yourtable where questionId = "+str(i))
        #如果上sql有记录，判断当前题目车型里是否含有当前车型，没有则将当前车型更新到当前题目，有则无动作
        CxCount=cur.execute("select * from yourtable where questionId = "+str(i)+" and chexing like '%"+str(chexing)+"%'")
        if idCount == 0:
        #if 1==1:
            if k['optionA'] == "正确":
                if k['answer'] == 32:
                    answer = "错误,"
                else:
                    answer = "正确,"
                try:
                    imgUrl = k['mediaContent']
                except:
                    imgUrl = '0'
                try:
                    explain = k['explain']
                except:
                    explain = '0'
                sql = "INSERT INTO `yourtable` (`chexing`, `kemu`, `zhangjieId`, `id`, `type`, `area1`, `area2`, `area3`, `area4`, `area5`, `region`, `title`, `content`, `imagesPath`, `musicPath`, `daan`, `score`, `tikuIndex`, `jieshi`, `beiyong`, `imagesWidth`, `imagesHeight`, `A`, `B`, `C`, `D`, `title2`, `xuanxiang2`, `changjingshifan1`, `changjingshifan2`,`questionId`) VALUES (%s, %s, '0', NULL, '判断题', NULL, NULL, NULL, NULL, NULL, NULL,%s, NULL, %s, NULL, %s, '1', '1', %s, '', NULL, NULL, NULL, NULL, NULL, NULL, '', '', '', '',%s);"
                print sql
                reCount = cur.execute(sql,(chexing,kemu,k['question'],imgUrl,answer,explain,k['questionId']))
                conn.commit()
                print '判断题：'+str(k['question']) + '已插入数据库' 
            else:
                if k['answer'] == 16:
                    answer = "A,"
                elif k['answer'] == 32:
                    answer = "B,"
                elif k['answer'] == 64:
                    answer = "C,"
                else:
                    answer = "D,"
                try:
                    imgUrl = k['mediaContent']
                except:
                    imgUrl = "0"
                try:
                    explain = k['explain']
                except:
                    explain = '0'
                if k['optionType'] == 2:
                    atype = '多选题'
                else:
                    atype = '单选题'
                sql = "INSERT INTO `yourtable` (`chexing`, `kemu`, `zhangjieId`, `id`, `type`, `area1`, `area2`, `area3`, `area4`, `area5`, `region`, `title`, `content`, `imagesPath`, `musicPath`, `daan`, `score`, `tikuIndex`, `jieshi`, `beiyong`, `imagesWidth`, `imagesHeight`, `A`, `B`, `C`, `D`, `title2`, `xuanxiang2`, `changjingshifan1`, `changjingshifan2`,`questionId`) VALUES (%s, %s, '0', NULL, %s, NULL, NULL, NULL, NULL, NULL, NULL,%s, NULL, %s, NULL, %s, '1', '1', %s, '1', '1', '1', %s, %s, %s, %s, '', '', '', '', %s);"
                #print chexing,kemu,atype,k['question'],imgUrl,answer,explain,k['optionA'],k['optionB'],k['optionC'],k['optionD'],k['questionId']
                reCount = cur.execute(sql,(chexing,kemu,atype,k['question'],imgUrl,answer,explain,k['optionA'],k['optionB'],k['optionC'],k['optionD'],k['questionId']))
                conn.commit()
                
                print str(atype)+'：'+str(k['question']) + '已插入数据库'
        else:
             
            if CxCount == 1:
                print str(i)+':数据库已有当前题目'
            else:
                try:
                    sql = "update `yourtable` set chexing = CONCAT(`chexing`,'"+str(chexing)+"') where questionId = "+str(i)
                    print sql
                    reCount = cur.execute(sql)
                except:
                    sql = "update `yourtable` set chexing = CONCAT(`chexing`,'"+str(chexing)+"') where questionId = "+str(i)
                    print sql
                    reCount = cur.execute(sql)
    a=a+1
    if a==2:
        break
print "完成记录:"+str(a)
conn.commit()
cur.close()
conn.close()

1
已连接数据库
[{u'optionType': 1, u'mediaHeight': 0, u'id': 18747, u'explain': u'<p>\u67092\u79cd\u8fdd\u6cd5\u884c\u4e3a\uff1a<br/>1\u3001\u6253\u7535\u8bdd<br/>2\u3001\u4e0d\u7cfb\u5b89\u5168\u5e26</p>', u'question': u'\u52a8\u753b1\u4e2d\u6709\u51e0\u79cd\u8fdd\u6cd5\u884c\u4e3a\uff1f', u'mediaType': 2, u'label': u'7.1.1.1', u'mediaContent': u'http://file.open.jiakaobaodian.com/tiku/res/909800.mp4', u'answer': 32, u'mediaWidth': 0, u'questionId': 909800, u'optionH': u'', u'optionF': u'', u'optionG': u'', u'optionD': u'\u56db\u79cd\u8fdd\u6cd5\u884c\u4e3a', u'optionE': u'', u'optionB': u'\u4e8c\u79cd\u8fdd\u6cd5\u884c\u4e3a', u'optionC': u'\u4e09\u79cd\u8fdd\u6cd5\u884c\u4e3a', u'optionA': u'\u4e00\u79cd\u8fdd\u6cd5\u884c\u4e3a', u'difficulty': 2, u'chapterId': 127, u'wrongRate': 0.09629586735307373, u'trueCount': 280471734, u'falseCount': 29886185}]
909800:数据库已有当前题目
[{u'optionType': 1, u'mediaHeight': 0, u'id': 18748, u'explain': u'<p>\u67092\u79cd\u8fdd\u6cd5\u884c\u4e3a\uff1a<br/>1\u30